In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
data = pd.read_csv("../CSV/big_files/ais_train.csv", sep="|")
#data["time"] = pd.to_datetime(data["time"])

boat = data[data["vesselId"] == "61e9f3a8b937134a3c4bfdf7"]
data.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3


In [4]:
#Cleaning data first
cog = data[data["cog"] > 360]
cog.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

def evaluate(model: str, y_pred, y_test, f):

    mse_latitude = mean_squared_error(y_test, y_pred)
    mse_longitude = mean_squared_error(y_test, y_pred)
    r2_latitude = r2_score(y_test, y_pred)
    r2_longitude = r2_score(y_test, y_pred)

    f.write(f"---- {model} Metrics ----\n")
    f.write(f"Mean Squared Error (Latitude): {mse_latitude:.4f}\n")
    f.write(f"R-squared (Latitude): {r2_latitude:.4f}\n")
    f.write(f"Mean Squared Error (Longitude): {mse_longitude:.4f}\n")
    f.write(f"R-squared (Longitude): {r2_longitude:.4f}\n")
    f.write("\n")

def powerset(seq):
    """
    Returns all the subsets of this set. This is a generator.
    """
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item


In [44]:
from xgboost import XGBRegressor
f = open("XGBoost results", "a")

boat = data[data["vesselId"] == "61e9f3a8b937134a3c4bfdf7"]
boat = boat.drop(columns=["vesselId", "portId", "etaRaw"])
boat["time"] = pd.to_datetime(boat["time"]).astype("int64") // (10**9)

y = boat[["latitude", "longitude"]]
boat.drop(columns=["latitude", "longitude"], inplace=True)

bruh = [x for x in powerset([1,2,3])]
bruh.sort()
print(bruh[1::])

    

[[1], [1, 2], [1, 2, 3], [1, 3], [2], [2, 3], [3]]


In [53]:
cols = [col for col in boat.columns[1::]]
wallahi = [x for x in powerset(cols)]
wallahi.sort()
wallahi = wallahi[1::]

for col in wallahi:
    X = boat[["time"] + col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    xgb = XGBRegressor()
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    evaluate(col, y_pred, y_test, X_test, f)

f.close()

In [42]:
from geopy.distance import distance

def speed(boat):
    boat = boat.drop(columns=["vesselId", "portId", "etaRaw"])
    boat.reset_index(inplace=True, drop=True)

    boat["time"] = pd.to_datetime(boat["time"])
    boat["time_shifted"] = boat["time"].shift(1)
    boat["delta_time_hr"] = boat["time"] - boat["time_shifted"]
    boat["delta_time_hr"].fillna(pd.Timedelta(0), inplace=True)
    boat["delta_time_hr"] = boat["delta_time_hr"].dt.total_seconds()/3600

    boat["lat_shift"] = boat["latitude"].shift(-1)
    boat["lon_shift"] = boat["longitude"].shift(-1)
    boat["speed"] = np.nan

    for i in boat.index[1:-1]:
        boat.at[i, "speed"] = distance((boat.at[i, "latitude"], boat.at[i, "longitude"]), (boat.at[i, "lat_shift"], boat.at[i, "lon_shift"])).km / boat.at[i, "delta_time_hr"]




In [44]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ids = data["vesselId"].unique()
wallah = []

for id in ids:
    boat = data[data["vesselId"] == id]

    wallah.append(speed(boat))

wallah = pd.DataFrame(wallah)
wallah.to_csv("speeds.csv", sep="|")


KeyboardInterrupt: 

In [2]:
speeds = pd.read_csv("speeds.csv", sep="|")
speeds.drop(columns=["Unnamed: 0"], inplace=True)
speeds.head()

,time,cog,sog,rot,heading,navstat,latitude,longitude,delta_time_hr,speed
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,-34.74370,-57.85130,0.000000,0.000000
1,2024-01-01 06:09:08,92.8,14.2,0,90,0,-35.16787,-56.77210,6.145278,2.079187
2,2024-01-01 06:38:19,90.5,14.3,0,88,0,-35.16863,-56.63185,0.486389,18.721138
3,2024-01-01 06:58:55,88.2,14.3,0,86,0,-35.16805,-56.53190,0.343333,20.921788
4,2024-01-01 07:15:56,88.3,12.3,0,86,0,-35.16715,-56.45306,0.283611,16.063438


In [5]:
speeds = speeds[speeds["speed"] < 50]
ais = pd.read_csv("../CSV/big_files/ais_train.csv", sep="|")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [7]:
speeds.sort_values("time", inplace=True)
speeds["vesselId"] = ais["vesselId"]
speeds.head()

,time,cog,sog,rot,heading,navstat,latitude,longitude,delta_time_hr,speed,vesselId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,-34.74370,-57.85130,0.0,0.0,61e9f3a8b937134a3c4bfdf7
1550,2024-01-01 00:00:36,109.6,0.0,-6,347,1,8.89440,-79.47939,0.0,0.0,61e9f438b937134a3c4c0145
3244,2024-01-01 00:01:45,111.0,11.0,0,112,0,39.19065,-76.47567,0.0,0.0,61e9f436b937134a3c4c0131
5080,2024-01-01 00:03:11,96.4,0.0,0,142,1,-34.41189,151.02067,0.0,0.0,61e9f46ab937134a3c4c02a9
6386,2024-01-01 00:03:51,214.0,19.7,0,215,0,35.88379,-5.91636,0.0,0.0,61e9f470b937134a3c4c02d1


In [8]:
y = speeds[["latitude", "longitude"]]
X = speeds[["time", "vesselId", "speed", "delta_time_hr"]]
X = pd.get_dummies(X, columns=["vesselId"], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

,time,vesselId,speed,delta_time_hr
0,2024-01-01 00:00:25,61e9f3a8b937134a3c4bfdf7,0.0,0.0
1550,2024-01-01 00:00:36,61e9f438b937134a3c4c0145,0.0,0.0
3244,2024-01-01 00:01:45,61e9f436b937134a3c4c0131,0.0,0.0
5080,2024-01-01 00:03:11,61e9f46ab937134a3c4c02a9,0.0,0.0
6386,2024-01-01 00:03:51,61e9f470b937134a3c4c02d1,0.0,0.0


In [ ]:
# --- LINEAR REGRESSION ---
import time
f = open("linear regression.txt", a)

start = time.time()
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)
print(time.time() - start)

evaluate("Time, Speed, Delta_time_hr", y_pred, y_test, f)
